In [32]:
from sklearn import datasets
from sklearn.model_selection import train_test_split


import pickle

In [ ]:
# 3.5
    # max_samples=0.5 dla 50% instancji

In [33]:
# method used for saving object as pickle
def save_object_as_pickle(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, pickle.HIGHEST_PROTOCOL)

## Import the data 

In [2]:
data_breast_cancer = datasets.load_breast_cancer(as_frame=True) 
print(data_breast_cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [6]:
# split the dataset
X = data_breast_cancer['data'][['mean texture', 'mean symmetry']]
y = data_breast_cancer['frame'].target
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 3 bazowe + voting hard + voting soft ---- tj. razem 5 elementow 

In [8]:
X_train.head()

,mean texture,mean symmetry
68,17.33,0.2111
181,26.57,0.2395
63,13.86,0.2341
248,25.22,0.1897
60,14.88,0.2743


In [9]:
y_train.head()

68     1
181    0
63     1
248    1
60     1
Name: target, dtype: int64

# Ensemble

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import VotingClassifier


from sklearn.metrics import accuracy_score

In [40]:
dtr_clf = DecisionTreeClassifier(random_state=42)
log_clf = LogisticRegression(random_state=42)
knn_clf = KNeighborsClassifier()

voting_clf_hard = VotingClassifier(
    estimators=[('lr', log_clf),
                ('dtr', dtr_clf),
                ('knn', knn_clf)],
    voting='hard')

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf),
                ('dtr', dtr_clf),
                ('knn', knn_clf)],
    voting='soft')

### Calculate accuracy

In [41]:
acc_list = []

for clf in (dtr_clf, log_clf, knn_clf, voting_clf_hard, voting_clf_soft):
    clf.fit(X_train, y_train)
    y_pred_test = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    print("TRAIN", clf.__class__.__name__,
      train_acc)
    print("TEST", clf.__class__.__name__,
          test_acc)
    acc_list.append((train_acc, test_acc))
    
# WARNING! First voting classifier has 'hard' type of voting
# second voting classifier has 'soft' type of voting (it's not the same classifier!)

TRAIN DecisionTreeClassifier 1.0
TEST DecisionTreeClassifier 0.6140350877192983
TRAIN LogisticRegression 0.7230769230769231
TEST LogisticRegression 0.7017543859649122
TRAIN KNeighborsClassifier 0.7714285714285715
TEST KNeighborsClassifier 0.6403508771929824
TRAIN VotingClassifier 0.8351648351648352
TEST VotingClassifier 0.6929824561403509
TRAIN VotingClassifier 0.9648351648351648
TEST VotingClassifier 0.6666666666666666


In [42]:
# test if data is properly saved
for tup in acc_list:
    print(tup)

(1.0, 0.6140350877192983)
(0.7230769230769231, 0.7017543859649122)
(0.7714285714285715, 0.6403508771929824)
(0.8351648351648352, 0.6929824561403509)
(0.9648351648351648, 0.6666666666666666)


### Save data in the pickles

In [43]:
save_object_as_pickle(acc_list, "acc_vote.pkl")

clf_list = [dtr_clf, log_clf, knn_clf, voting_clf_hard, voting_clf_soft]

save_object_as_pickle(clf_list, "vote.pkl")

### Check if data saved properly

## 30 decision trees:
* Bagging,
* Bagging z wykorzystaniem 50% instancji,
* Pasting,
* Pasting z wykorzystaniem 50% instancji, oraz 
* Random Forest,
* AdaBoost,
* Gradient Boosting.


In [46]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

In [48]:
bgn_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, random_state=42)
rnf_clf = RandomForestClassifier(n_estimators=30, random_state=42)

In [50]:
# Odnosnie zad 7: Tutaj BaggingClassifier z decision tree i max_features=2, 
# reszta ustawien domyslna trzeba uzyc